In [ ]:
@patch("rss_collector.utils.get_proxy_settings")
@patch("rss_collector.proxy.fetch_feed_content_with_proxy")
@patch("rss_collector.feed_parser.parse_feed")
def test_process_feeds(
    mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings, test_feed_urls
):
    print("=== Starting test_process_feeds ===")

    # Mock proxy settings
    mock_get_proxy_settings.return_value = mock_proxy_settings
    print("Mocked get_proxy_settings return value:", mock_proxy_settings)

    # Mock fetch_feed_content_with_proxy to return mock feed content
    mock_fetch_feed.return_value = "Mock RSS Feed Content"
    print("Mocked fetch_feed_content_with_proxy return value: 'Mock RSS Feed Content'")

    # Mock parse_feed to return the mocked feed data
    mock_parse_feed.return_value = mock_feed_data
    print("Mocked parse_feed return value:", mock_feed_data)

    # Call the process_feeds function
    process_feeds(test_feed_urls)

    print("=== Finished process_feeds call ===")

    # Assertions for get_proxy_settings
    mock_get_proxy_settings.assert_called_once()
    print("get_proxy_settings called once")

    # Assertions for fetch_feed_content_with_proxy
    assert mock_fetch_feed.call_count == len(test_feed_urls)
    print("fetch_feed_content_with_proxy called for each feed URL")

    # Assertions for parse_feed
    assert mock_parse_feed.call_count == len(test_feed_urls)
    print("parse_feed called for each feed URL")

    # Verify JSON file creation
    output_dir = "rss_metadata"
    saved_files = os.listdir(output_dir)
    print("Saved files in output_dir:", saved_files)
    assert len(saved_files) == 1  # Only one consolidated JSON file should be created

    # Validate JSON content
    output_file = os.path.join(output_dir, saved_files[0])
    with open(output_file, "r") as f:
        metadata = json.load(f)

    assert "timestamp" in metadata
    assert "articles" in metadata
    assert len(metadata["articles"]) == len(mock_feed_data["entries"])
    print("Validated JSON content structure")

    # Cleanup created files
    for file in saved_files:
        os.remove(os.path.join(output_dir, file))
    os.rmdir(output_dir)
    print("Cleaned up output directory")
